# 构建基于PPI的基础模型

In [1]:
import optuna
from optuna.samplers import TPESampler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, train_test_split
import xgboost as xgb
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

np.random.seed(42)

# 加载数据集

data1 = pd.read_csv('ppi_10.csv')
X = data1.iloc[:, 1:]
y = data1.iloc[:, 0]  # 假设标签在 data1 的第一列

# 数据标准化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 拆分数据集
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)



In [2]:
# 5折交叉验证
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 目标函数
def objective(trial, model_name):
    if model_name == 'et':
        model = ExtraTreesClassifier(
            max_depth=trial.suggest_int('max_depth', 5, 20),
            min_samples_split=trial.suggest_int('min_samples_split', 2, 20),
            n_estimators=trial.suggest_int('n_estimators', 50, 300),
            random_state=42
        )
    elif model_name == 'knn':
        model = KNeighborsClassifier(
            n_neighbors=trial.suggest_int('n_neighbors', 3, 20),
            leaf_size=trial.suggest_int('leaf_size', 5, 50),
            weights=trial.suggest_categorical('weights', ['uniform', 'distance'])
        )
    elif model_name == 'logistic':
        model = LogisticRegression(
            C=trial.suggest_float('C', 1e-4, 1e2, log=True),
            penalty='l1',
            solver='saga',
            random_state=42
        )
    elif model_name == 'xgb':
        model = xgb.XGBClassifier(
            learning_rate=trial.suggest_float('learning_rate', 0.01, 0.3),
            max_depth=trial.suggest_int('max_depth', 3, 10),
            n_estimators=trial.suggest_int('n_estimators', 50, 300),
            use_label_encoder=False,
            eval_metric='logloss',
            random_state=42
        )
    elif model_name == 'rf':
        model = RandomForestClassifier(
            max_depth=trial.suggest_int('max_depth', 5, 20),
            min_samples_split=trial.suggest_int('min_samples_split', 2, 20),
            n_estimators=trial.suggest_int('n_estimators', 50, 300),
            random_state=42
        )
    elif model_name == 'gnb':
        model = GaussianNB(
            var_smoothing=trial.suggest_float('var_smoothing', 1e-12, 1e-7, log=True)
        )
    elif model_name == 'lda':
        model = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
    elif model_name == 'ada':
        model = AdaBoostClassifier(
            learning_rate=trial.suggest_float('learning_rate', 0.01, 2),
            n_estimators=trial.suggest_int('n_estimators', 50, 300),
            algorithm='SAMME',
            random_state=42
        )
    elif model_name == 'gb':
        model = GradientBoostingClassifier(
            learning_rate=trial.suggest_float('learning_rate', 0.01, 0.3),
            max_depth=trial.suggest_int('max_depth', 3, 10),
            min_samples_split=trial.suggest_int('min_samples_split', 2, 20),
            n_estimators=trial.suggest_int('n_estimators', 50, 300),
            subsample=trial.suggest_float('subsample', 0.5, 1.0),
            random_state=42
        )
    elif model_name == 'svm':
        model = SVC(
            C=trial.suggest_float('C', 1e-3, 1e2, log=True),
            probability=True,
            random_state=42
        )

    # 5折交叉验证计算 AUC 均值
    auc_scores = []
    for train_idx, val_idx in cv.split(X_train, y_train):
        X_tr, X_val = X_train[train_idx], X_train[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

        model.fit(X_tr, y_tr)
        y_pred = model.predict_proba(X_val)[:, 1]
        auc_scores.append(roc_auc_score(y_val, y_pred))

    return np.mean(auc_scores)

# 运行 Optuna 进行超参数优化
optimized_models = {}
best_aucs = {}

for model_name in ['et', 'knn', 'logistic', 'xgb', 'rf', 'gnb', 'lda', 'ada', 'gb', 'svm']:
    study = optuna.create_study(direction='maximize', sampler=TPESampler())
    study.optimize(lambda trial: objective(trial, model_name), n_trials=50)

    optimized_models[model_name] = study.best_params
    best_aucs[model_name] = study.best_value

    print(f"Best AUC for {model_name}: {study.best_value}")
    print(f"Best parameters for {model_name}: {study.best_params}")

# 输出每个模型的最佳 AUC
print("\nModel Best AUCs:")
for model_name, auc in best_aucs.items():
    print(f"{model_name}: {auc}")
    



[I 2025-04-03 23:10:04,623] A new study created in memory with name: no-name-18c26293-8811-40fe-a2e4-cbc5157002e3
[I 2025-04-03 23:10:05,528] Trial 0 finished with value: 0.6759413992869876 and parameters: {'max_depth': 16, 'min_samples_split': 16, 'n_estimators': 296}. Best is trial 0 with value: 0.6759413992869876.
[I 2025-04-03 23:10:06,291] Trial 1 finished with value: 0.6921290106951872 and parameters: {'max_depth': 11, 'min_samples_split': 5, 'n_estimators': 247}. Best is trial 1 with value: 0.6921290106951872.
[I 2025-04-03 23:10:06,599] Trial 2 finished with value: 0.6871323529411766 and parameters: {'max_depth': 17, 'min_samples_split': 8, 'n_estimators': 99}. Best is trial 1 with value: 0.6921290106951872.
[I 2025-04-03 23:10:07,149] Trial 3 finished with value: 0.6986965240641712 and parameters: {'max_depth': 18, 'min_samples_split': 2, 'n_estimators': 171}. Best is trial 3 with value: 0.6986965240641712.
[I 2025-04-03 23:10:07,390] Trial 4 finished with value: 0.68843025846

[I 2025-04-03 23:10:26,758] Trial 40 finished with value: 0.6917780748663102 and parameters: {'max_depth': 15, 'min_samples_split': 5, 'n_estimators': 270}. Best is trial 28 with value: 0.7057486631016043.
[I 2025-04-03 23:10:27,358] Trial 41 finished with value: 0.6842858734402852 and parameters: {'max_depth': 18, 'min_samples_split': 3, 'n_estimators': 179}. Best is trial 28 with value: 0.7057486631016043.
[I 2025-04-03 23:10:27,859] Trial 42 finished with value: 0.6937305035650624 and parameters: {'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 151}. Best is trial 28 with value: 0.7057486631016043.
[I 2025-04-03 23:10:28,524] Trial 43 finished with value: 0.6926860516934046 and parameters: {'max_depth': 17, 'min_samples_split': 4, 'n_estimators': 202}. Best is trial 28 with value: 0.7057486631016043.
[I 2025-04-03 23:10:28,708] Trial 44 finished with value: 0.6779133244206774 and parameters: {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 52}. Best is trial 28 wit

Best AUC for et: 0.7057486631016043
Best parameters for et: {'max_depth': 19, 'min_samples_split': 4, 'n_estimators': 143}


[I 2025-04-03 23:10:31,551] Trial 15 finished with value: 0.6296847147950089 and parameters: {'n_neighbors': 14, 'leaf_size': 33, 'weights': 'distance'}. Best is trial 10 with value: 0.6529133244206774.
[I 2025-04-03 23:10:31,567] Trial 16 finished with value: 0.6222398618538325 and parameters: {'n_neighbors': 3, 'leaf_size': 18, 'weights': 'distance'}. Best is trial 10 with value: 0.6529133244206774.
[I 2025-04-03 23:10:31,585] Trial 17 finished with value: 0.6529133244206774 and parameters: {'n_neighbors': 8, 'leaf_size': 6, 'weights': 'distance'}. Best is trial 10 with value: 0.6529133244206774.
[I 2025-04-03 23:10:31,602] Trial 18 finished with value: 0.6429924242424242 and parameters: {'n_neighbors': 20, 'leaf_size': 18, 'weights': 'distance'}. Best is trial 10 with value: 0.6529133244206774.
[I 2025-04-03 23:10:31,617] Trial 19 finished with value: 0.6245933600713013 and parameters: {'n_neighbors': 15, 'leaf_size': 31, 'weights': 'distance'}. Best is trial 10 with value: 0.652913

[I 2025-04-03 23:10:32,199] Trial 1 finished with value: 0.5 and parameters: {'C': 0.00025582219429886346}. Best is trial 0 with value: 0.6750111408199643.
[I 2025-04-03 23:10:32,211] Trial 2 finished with value: 0.46951871657754013 and parameters: {'C': 0.06310545868356429}. Best is trial 0 with value: 0.6750111408199643.
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coe

...

[I 2025-04-03 23:11:51,707] Trial 0 finished with value: 0.6690006684491978 and parameters: {'learning_rate': 0.13702653811920046, 'max_depth': 10, 'min_samples_split': 11, 'n_estimators': 101, 'subsample': 0.7091554599643576}. Best is trial 0 with value: 0.6690006684491978.
[I 2025-04-03 23:11:52,548] Trial 1 finished with value: 0.6379177807486631 and parameters: {'learning_rate': 0.195832018093705, 'max_depth': 3, 'min_samples_split': 19, 'n_estimators': 243, 'subsample': 0.803607263179579}. Best is trial 0 with value: 0.6690006684491978.
[I 2025-04-03 23:11:54,381] Trial 2 finished with value: 0.6848707664884135 and parameters: {'learning_rate': 0.04073104058180706, 'max_depth': 9, 'min_samples_split': 6, 'n_estimators': 299, 'subsample': 0.6251469955080247}. Best is trial 2 with value: 0.6848707664884135.
[I 2025-04-03 23:11:54,761] Trial 3 finished with value: 0.6462622549019608 and parameters: {'learning_rate': 0.14478643513321154, 'max_depth': 3, 'min_samples_split': 2, 'n_esti

[I 2025-04-03 23:12:22,045] Trial 30 finished with value: 0.6810494652406416 and parameters: {'learning_rate': 0.16684658718690332, 'max_depth': 7, 'min_samples_split': 4, 'n_estimators': 188, 'subsample': 0.8822241909984181}. Best is trial 21 with value: 0.698117201426025.
[I 2025-04-03 23:12:23,444] Trial 31 finished with value: 0.6843192959001783 and parameters: {'learning_rate': 0.23528677078516747, 'max_depth': 8, 'min_samples_split': 2, 'n_estimators': 278, 'subsample': 0.8192903490815826}. Best is trial 21 with value: 0.698117201426025.
[I 2025-04-03 23:12:24,714] Trial 32 finished with value: 0.6864471925133689 and parameters: {'learning_rate': 0.24874648119698736, 'max_depth': 10, 'min_samples_split': 3, 'n_estimators': 252, 'subsample': 0.7837891992656376}. Best is trial 21 with value: 0.698117201426025.
[I 2025-04-03 23:12:25,991] Trial 33 finished with value: 0.6913213012477718 and parameters: {'learning_rate': 0.2766081914712293, 'max_depth': 8, 'min_samples_split': 2, 'n_

Best AUC for gb: 0.7144607843137254
Best parameters for gb: {'learning_rate': 0.27824447470120994, 'max_depth': 9, 'min_samples_split': 3, 'n_estimators': 272, 'subsample': 0.8850316686412799}


[I 2025-04-03 23:12:43,433] Trial 9 finished with value: 0.6738803475935828 and parameters: {'C': 0.8464474159823367}. Best is trial 2 with value: 0.6828988413547237.
[I 2025-04-03 23:12:43,456] Trial 10 finished with value: 0.6826815953654191 and parameters: {'C': 0.009422981089100854}. Best is trial 2 with value: 0.6828988413547237.
[I 2025-04-03 23:12:43,478] Trial 11 finished with value: 0.6818293226381462 and parameters: {'C': 0.06946219072148478}. Best is trial 2 with value: 0.6828988413547237.
[I 2025-04-03 23:12:43,499] Trial 12 finished with value: 0.6806929590017825 and parameters: {'C': 0.03231779515440809}. Best is trial 2 with value: 0.6828988413547237.
[I 2025-04-03 23:12:43,525] Trial 13 finished with value: 0.645727495543672 and parameters: {'C': 5.200303244452213}. Best is trial 2 with value: 0.6828988413547237.
[I 2025-04-03 23:12:43,546] Trial 14 finished with value: 0.6806929590017825 and parameters: {'C': 0.034255814766687015}. Best is trial 2 with value: 0.6828988

Best AUC for svm: 0.6860405525846702
Best parameters for svm: {'C': 0.38995453773975636}

Model Best AUCs:
et: 0.7057486631016043
knn: 0.6558294340463459
logistic: 0.6935606060606061
xgb: 0.6904188948306597
rf: 0.6883021390374331
gnb: 0.6926526292335116
lda: 0.6904244652406417
ada: 0.6574448529411765
gb: 0.7144607843137254
svm: 0.6860405525846702


In [3]:
import joblib
from itertools import combinations
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier

# 使用最佳参数初始化模型
optimized_models_instances = {
    'et': ExtraTreesClassifier(**optimized_models['et'], random_state=42),
    'knn': KNeighborsClassifier(**optimized_models['knn']),
    'logistic': LogisticRegression(**optimized_models['logistic'], random_state=42),
    'xgb': xgb.XGBClassifier(**optimized_models['xgb'], random_state=42),
    'rf': RandomForestClassifier(**optimized_models['rf'], random_state=42),
    #'bagging': BaggingClassifier(**optimized_models['bagging'], random_state=42),
    'gnb': GaussianNB(**optimized_models['gnb']),
    'lda': LinearDiscriminantAnalysis(**optimized_models['lda']),
    'ada': AdaBoostClassifier(**optimized_models['ada'], random_state=42, algorithm='SAMME'),
    'gb': GradientBoostingClassifier(**optimized_models['gb'], random_state=42),
    'svm': SVC(**optimized_models['svm'], probability=True, random_state=42)
}

# 保存每个优化后的模型
for model_name, model in optimized_models_instances.items():
    joblib.dump(model, f'C:\\Users\\一个大活人\\ppi-wgcna-sssgsea-4.3\\basemodel\\ppi\\{model_name}_ppi.joblib')  
    print(f"Model {model_name} saved successfully.")



Model et saved successfully.
Model knn saved successfully.
Model logistic saved successfully.
Model xgb saved successfully.
Model rf saved successfully.
Model gnb saved successfully.
Model lda saved successfully.
Model ada saved successfully.
Model gb saved successfully.
Model svm saved successfully.
